In [29]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from pandasql import sqldf
import json
from collections import defaultdict

Acceptance rate per question (in leetcode format)

In [33]:
print("Which Contest would you like to view?")
print()
contest = input()
page = "https://codeforces.com/api/contest.status?contestId=" + contest + "&from=1"
cf_submissions_api = requests.get(page)
submissions = cf_submissions_api.json()
cf_dataframe = pd.json_normalize(submissions,['result'])

#fetch the duration of the contest
cf_duration_api = requests.get("https://codeforces.com/api/contest.list?gym=false")
contests = cf_duration_api.json()
cf_duration_dataframe = pd.json_normalize(contests,['result'])

contest_name = list(cf_duration_dataframe[cf_duration_dataframe["id"]== 1610].name)[0]
print("You are viewing " + contest_name)
print()

selected_contest = cf_duration_dataframe.loc[cf_duration_dataframe["id"] == int(contest)]
start_time = list(selected_contest["startTimeSeconds"])[0]
duration = list(selected_contest["durationSeconds"])[0]

#enter time after the contest started (in seconds) to view stats
#for example: if you want to see acceptance rate 5 minutes into the contest, key in 300
print("Duration after contest started ?")
print()
time_after_start = int(input())
time_after_start = min(time_after_start,duration)
print("You are viewing acceptance rate " + str(time_after_start) + " seconds into the contest")
print()

end_time = list(selected_contest["startTimeSeconds"])[0] + time_after_start
byTimeMemory = cf_dataframe[["creationTimeSeconds","contestId","problem.index","problem.name","programmingLanguage","author.members","verdict","timeConsumedMillis", "memoryConsumedBytes"]].loc[(cf_dataframe["creationTimeSeconds"] >= start_time) & (cf_dataframe["creationTimeSeconds"] <= end_time)]
byTimeMemory = byTimeMemory.rename(columns = {"timeConsumedMillis": "Time","problem.index":"problem_index","problem.name":"problem_name","author.members":"author_members"})

#retrieve the name and id of the problems
problem_header = byTimeMemory[["problem_index","problem_name"]]
problems = sorted(set(list(byTimeMemory["problem_index"])))
index_to_name = defaultdict()
for index,name in problem_header.values:
    index_to_name[index] = name
    if len(index_to_name) == len(problems):
        break
    
for pindex in problems:
    solved_df = byTimeMemory[(byTimeMemory["problem_index"] == pindex) & (byTimeMemory["verdict"] == "OK")]
    attempted_df = byTimeMemory[(byTimeMemory["problem_index"] == pindex)]
    user_solved = len(set([x[0]["handle"] for x in list(solved_df["author_members"])]))
    user_attempted = len(set([x[0]["handle"] for x in list(attempted_df["author_members"])]))
    total_solved = len(solved_df)
    total_attempted = len(attempted_df)
    acceptance_rate = ((total_solved / total_attempted) * 100)
    print(f"{contest}{pindex}. {index_to_name[pindex]}")
    print(f"User Accepted: {user_solved}")
    print(f"User Tried: {user_attempted}")
    print(f"Total Accepted: {total_solved}")
    print(f"Total Submissions: {total_attempted}")
    print(f"Acceptance Rate: {acceptance_rate} % ")
    print("-----------------------------------------------------------------------------------")

Which Contest would you like to view?

You are viewing Codeforces Global Round 17

Duration after contest started ?

You are viewing acceptance rate 180 seconds into the contest

1610A. Anti Light's Cell Guessing
User Accepted: 258
User Tried: 1040
Total Accepted: 258
Total Submissions: 1156
Acceptance Rate: 22.318339100346023 % 
-----------------------------------------------------------------------------------
